In [6]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [15]:
df = pd.read_excel('Dataset\\Online_retail.xlsx')

# Rimuoviamo gli spazi vuoti dai nomi delle colonne
df.columns = df.columns.str.strip()

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
df['Quantity'] = df['Quantity'].abs()

# Uniamo i prodotti per ciascuna transazione (basato su InvoiceNo)
basket = (df.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Codifica dei dati in formato one-hot encoding
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

# Applicazione dell'algoritmo Apriori per trovare gli itemset frequenti
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)

# Generazione delle regole di associazione
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Rimuovi le regole duplicate considerando solo una direzione (es. A -> B)
rules['antecedents'] = rules['antecedents'].apply(lambda x: next(iter(x)))
rules['consequents'] = rules['consequents'].apply(lambda x: next(iter(x)))

# Rimuovi regole duplicate
rules = rules.drop_duplicates(['antecedents', 'consequents'])

# Visualizza le regole di associazione
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


C:\Users\arrag\AppData\Local\Temp\ipykernel_95724\3509950329.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.applymap(encode_units)
C:\Users\arrag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


                        antecedents                       consequents  \
0   GREEN REGENCY TEACUP AND SAUCER  ROSES REGENCY TEACUP AND SAUCER    
1  ROSES REGENCY TEACUP AND SAUCER    GREEN REGENCY TEACUP AND SAUCER   
2           JUMBO BAG PINK POLKADOT           JUMBO BAG RED RETROSPOT   
3           JUMBO BAG RED RETROSPOT           JUMBO BAG PINK POLKADOT   

    support  confidence       lift  
0  0.032071    0.741722  16.189404  
1  0.032071    0.700000  16.189404  
2  0.034075    0.676686   7.748130  
3  0.034075    0.390164   7.748130  
